# Set up and mount drives

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%cd /content/drive/'My Drive'/Colab_Notebooks/knbs_2019/

In [54]:
ls


kenya_census_2019/  Kenya_subCounty.csv


In [56]:
ls ./kenya_census_2019

'Kenya Average Household Size by sub-county.csv'
'Kenya Population by sex and county.csv'
'Kenya Population by Sub-County.csv'
'Kenya Population, Land area & Population density_by_County.csv'
'Kenya Population,Land area & Population density_by_subCounty.csv'
'Population by Land area.csv'
'Population & Houseshold data.csv'


## Import neccessities 

In [0]:
# !pip install ckanapi

In [0]:
import sys
import os

import ckanapi
import requests
# import api_key

import pandas as pd

## Credentials and Working folder

In [0]:
data_portal = 'https://africaopendata.org'

# upload api
api_url = data_portal + '/api/action/resource_create'

# APIKEY = api_key.token
APIKEY = 'xxxkjgggjj5663993666363xxx'

ckan = ckanapi.RemoteCKAN(data_portal, apikey=APIKEY)

headers = {'Authorization': APIKEY}

dataset_url = '2019-kenya-population-and-housing-census'

dataset_title = '2019 Kenya Population and Housing Census'

datasets_folder = './kenya_census_2019'

## Create Package  / host Repo for the datasets

In [57]:
# Create the "Data viewer examples" dataset.
package_name = dataset_url
package_title = dataset_title
try:
    print ('Creating "{package_title}" package'.format(**locals()))
    package = ckan.action.package_create(name=package_name,
                                         title=package_title,
                                        owner_org = 'knbs')
except (ckanapi.ValidationError) as e:
    if (e.error_dict['__type'] == 'Validation Error' and
       e.error_dict['name'] == ['That URL is already in use.']):
        print ('"{package_title}" package already exists'.format(**locals()))
        package = ckan.action.package_show(id=package_name)
    else:
        raise

Creating "2019 Kenya Population and Housing Census" package
"2019 Kenya Population and Housing Census" package already exists


## Upload Datasets  / host Repo for the datasets

In [52]:
ls datasets_folder

ls: cannot access 'datasets_folder': No such file or directory


In [58]:
for item in os.listdir(datasets_folder):
    path = os.path.join(datasets_folder,item)
    extension = os.path.splitext(item)[1][1:].lower()
    filename = os.path.splitext(item)[0][0:].lower()
    resource_name = item,'{extension}'.format(extension=extension)
    data_file= pd.read_csv(path)
    data_desc = data_file[:3].dropna(axis=1)
    desc = ', '.join(data_desc["Unnamed: 0"])
  
    print ('Creating "{filename}"'.format(**locals()))
    r = requests.post(api_url,
                      data={'package_id': package['id'],
                            'name': filename,
                            'format': extension,
                            'description': desc
#                             'url': item,  # Needed to pass validation
                            },
                      headers={'Authorization': APIKEY},
#                       files=[('upload', file('"{path}"'.format(path=path)))])
                      files=[('upload', open(path))])

    if r.status_code != 200:
        print ('Error while creating resource: {0}'.format(r.content))
        break
    

Creating "kenya population by sub-county"
Creating "kenya population by sex and county"
Creating "kenya average household size by sub-county"
Creating "kenya population,land area & population density_by_subcounty"
Creating "population & houseshold data"
Creating "population by land area"
Creating "kenya population, land area & population density_by_county"
